# Data Merger

In this notebook, we will merge the statistics (per game and advanced), salary data, and transaction data.

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from datetime import datetime, timedelta
from nba_api.stats.static import teams

In [2]:
# This ensures side effects do not occur when dealing with views and copies of
# dataframes. See
# https://pandas.pydata.org/pandas-docs/stable//user_guide/copy_on_write.html.

pd.options.mode.copy_on_write = True

In [3]:
pd.options.display.max_columns = None

### Load data

In [4]:
stats_df = pd.read_csv('./bbref_stats.csv')

In [5]:
transaction_df = pd.read_csv('../transaction_data/transaction_data.csv')

# rename team column of transaction_df, drop nba id
transaction_df = transaction_df.rename(columns={'TEAM_ABBREVIATION': 'TEAM'})
transaction_df = transaction_df.drop(columns=['PLAYER_ID'])

In [6]:
nba_key_dates = pd.read_csv('../merged_data/data_to_merge/nba_key_dates.csv')

# remove dates prior to 1989-1990 season
nba_key_dates = nba_key_dates[nba_key_dates['SEASON_START']>=1989]

In [7]:
salary_df = pd.read_csv('./bbref_salary_data.csv')

# remove salaries prior to 1990-91 season
salary_df = salary_df[salary_df['SEASON_START']>=1990]

### Fix team abbreviations

In [8]:
nbs_api_teams = [team["abbreviation"] for team in teams.get_teams()]

In [9]:
print('Teams in stats_df not found in nba_api:')
for team_abv in stats_df["TEAM"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in transaction_df not found in nba_api:')
for team_abv in transaction_df["TEAM"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in salary_df not found in nba_api:')
for team_abv in salary_df["TEAM"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

Teams in stats_df not found in nba_api:
WSB
PHO
TOT
SEA
NJN
CHH
VAN
NOH
NOK
BRK
CHO

Teams in transaction_df not found in nba_api:

Teams in salary_df not found in nba_api:
WSB
PHO
SEA
CHH
VAN
NJN
NOH
NOK
BRK
CHO


In [10]:
team_abv_change_dict = {'BRK': 'BKN',
                        'CHH': 'CHA',
                        'CHO': 'CHA',
                        'GOS': 'GSW',
                        'NJN': 'BKN',
                        'NOH': 'NOP',
                        'NOK': 'NOP',
                        'PHL': 'PHI',
                        'PHO': 'PHX',
                        'SAN': 'SAS',
                        'SEA': 'OKC',
                        'UTH': 'UTA',
                        'VAN': 'MEM',
                        'WSB': 'WAS'}

In [11]:
stats_df['TEAM'] = stats_df['TEAM'].replace(team_abv_change_dict)
salary_df['TEAM'] = salary_df['TEAM'].replace(team_abv_change_dict)

In [12]:
# recheck team abbreviations

print('Teams in stats_df not found in nba_api:')
for team_abv in stats_df["TEAM"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in transaction_df not found in nba_api:')
for team_abv in transaction_df["TEAM"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in salary_df not found in nba_api:')
for team_abv in salary_df["TEAM"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

Teams in stats_df not found in nba_api:
TOT

Teams in transaction_df not found in nba_api:

Teams in salary_df not found in nba_api:


### Cleaning `transaction_df`

In [13]:
transaction_df = transaction_df[['BBREF_ID', 'SEASON_START', 'DATE', 'TEAM', 'WAIVED', 'RELEASED', 'TRADED']]
transaction_df = transaction_df.rename(columns={'BBREF_ID': 'PLAYER_ID'})

In [14]:
transaction_df

,PLAYER_ID,SEASON_START,DATE,TEAM,WAIVED,RELEASED,TRADED
0,bowiean01,1990,"July 5, 1990",HOU,1,0,0
1,allenra01,1990,"July 9, 1990",SAC,1,0,0
2,jacksmi02,1990,"July 9, 1990",SAC,1,0,0
3,turpime01,1990,"July 24, 1990",WAS,1,0,0
4,haffnsc01,1990,"August 10, 1990",MIA,1,0,0
...,...,...,...,...,...,...,...
8202,motiedo01,2016,"December 15, 2016",HOU,0,1,0
8203,toupaax01,2016,"March 4, 2017",MIL,0,1,0
8204,jacksde01,2017,"January 6, 2018",HOU,0,1,0
8205,nunnaja01,2018,"January 21, 2019",HOU,0,1,0


We will use/create the following columns:

- WAIVED_OFF
    - 1 if player was waived during this offseason, 0 otherwise.
- WAIVED_REG
    - 1 if player was waived during this regular season, 0 otherwise.
- WAIVED_POST
    - 1 if player was waived during this postseason, 0 otherwise.
- WAIVED
    - 1 if player was waived during this offseason, regular season, or postseason, 0 otherwise.
- WAIVED_NEXT_OFF
    - 1 if player was waived during the next offseason, 0 otherwise.
- WAIVED_NEXT_REG
    - 1 if player was waived during the next regular season, 0 otherwise.
- WAIVED_NEXT_POST
    - 1 if player was waived during the next postseason, 0 otherwise.
- WAIVED_NEXT
    - 1 if player was waived during the next offseason, regular season, or postseason, 0 otherwise.

Same as above for RELEASED and TRADED.

The definition I am using for the offseason is the first day after the last NBA finals game of the PREVIOUS SEASON to the day before the first regular season game. For example, the 2022 offseason was from 2022-06-17 to 2022-10-17.

The definition I am using for postseason is the first day after the last regular season game to the last day of the NBA finals.

To create these columns we will need to read the transaction dates and place them into the appropriate columns. To do this sorting, we will need to know the dates of each regular, post, and off-seasons. I have compiled this in nba_key_dates.csv.

We will also create a new column IN_LEAGUE_NEXT which yields a 1 if we have an entry for that player in our stats data for the following season, and 0 otherwise. If we don't consider this column, some misleading analysis could occur. For example, if we have no 0s for being waived in the following season, it could simply because they were not even in the league, and not because they are performing well.

First let's adjust the SEASON_START column of `transaction_data` to reflect the actual NBA season (in other words, if a transaction with a particular SEASON_START occurs after the end of the postseason, we add one to the SEASON_START since we actually want to consider it as occuring in the offseason of the *following* season).

For example, notice how rows 83 and 84 are for transactions occuring during the offseason of 1991, so we want to change SEASON_START for those rows to 1991.

In [15]:
transaction_df.iloc[82:86]

,PLAYER_ID,SEASON_START,DATE,TEAM,WAIVED,RELEASED,TRADED
82,schefst01,1990,"May 31, 1991",CHA,1,0,0
83,bannike01,1990,"June 21, 1991",LAC,1,0,0
84,wingada01,1990,"June 28, 1991",SAS,1,0,0
85,keysra01,1991,"July 15, 1991",CHA,1,0,0


In [16]:
# define a function which will return the updated SEASON_START for a given row

def update_season_start(row):
    date = datetime.strptime(row["DATE"], '%B %d, %Y').date()
    season_start = row["SEASON_START"]
    post_end = datetime.strptime(
        nba_key_dates.loc[nba_key_dates['SEASON_START'] == season_start, 'POST_END'].iloc[0],
        '%Y-%m-%d'
    ).date()
    if date > post_end:
        return season_start + 1
    return season_start

In [17]:
# apply the above function to the rows of transaction_data

transaction_df["SEASON_START"] = transaction_df.apply(update_season_start, axis=1)

Notice that SEASON_START for rows 83 and 84 have been updated correctly.

In [18]:
transaction_df.iloc[82:86]

,PLAYER_ID,SEASON_START,DATE,TEAM,WAIVED,RELEASED,TRADED
82,schefst01,1990,"May 31, 1991",CHA,1,0,0
83,bannike01,1991,"June 21, 1991",LAC,1,0,0
84,wingada01,1991,"June 28, 1991",SAS,1,0,0
85,keysra01,1991,"July 15, 1991",CHA,1,0,0


Now we add the new columns relating to whether a player is moved at some point during the given season.

In [19]:
for col in ['WAIVED_OFF', 'WAIVED_REG', 'WAIVED_POST',
            'RELEASED_OFF', 'RELEASED_REG', 'RELEASED_POST',
            'TRADED_OFF', 'TRADED_REG', 'TRADED_POST']:
    transaction_df[col] = 0

In [20]:
# define a function which returns the adds a 1 to the appropriate column for a given row

def set_transaction_type(row):
    if row['WAIVED'] == 1:
        transaction_type = 'WAIVED'
    elif row['RELEASED'] == 1:
        transaction_type = 'RELEASED'
    else:
        transaction_type = 'TRADED'

    date = datetime.strptime(row['DATE'], '%B %d, %Y').date()
    season_start = row['SEASON_START']

    reg_start = datetime.strptime(
        nba_key_dates.loc[nba_key_dates['SEASON_START']==season_start, 'REG_START'].iloc[0],
        '%Y-%m-%d').date()
    post_start = datetime.strptime(
        nba_key_dates.loc[nba_key_dates['SEASON_START'] == season_start, 'REG_END'].iloc[0],
        '%Y-%m-%d').date() + timedelta(days=1)

    if date < reg_start:
        row[f'{transaction_type}_OFF'] = 1
    elif date < post_start:
        row[f'{transaction_type}_REG'] = 1
    else:
        row[f'{transaction_type}_POST'] = 1

    return row

In [21]:
# apply the above function to the rows of transaction_data

transaction_df = transaction_df.apply(set_transaction_type, axis=1)

In [22]:
# rearrange columns

transaction_df = transaction_df[['PLAYER_ID', 'SEASON_START', 'TEAM', 'DATE',
                                 'WAIVED_OFF', 'WAIVED_REG', 'WAIVED_POST', 'WAIVED',
                                 'RELEASED_OFF', 'RELEASED_REG', 'RELEASED_POST', 'RELEASED',
                                 'TRADED_OFF', 'TRADED_REG', 'TRADED_POST', 'TRADED']]

In [23]:
transaction_df.iloc[79:86]

,PLAYER_ID,SEASON_START,TEAM,DATE,WAIVED_OFF,WAIVED_REG,WAIVED_POST,WAIVED,RELEASED_OFF,RELEASED_REG,RELEASED_POST,RELEASED,TRADED_OFF,TRADED_REG,TRADED_POST,TRADED
79,wilsotr01,1990,ATL,"April 1, 1991",0,1,0,1,0,0,0,0,0,0,0,0
80,longjo01,1990,DET,"April 5, 1991",0,1,0,1,0,0,0,0,0,0,0,0
81,greenda01,1990,SAS,"May 21, 1991",0,0,1,1,0,0,0,0,0,0,0,0
82,schefst01,1990,CHA,"May 31, 1991",0,0,1,1,0,0,0,0,0,0,0,0
83,bannike01,1991,LAC,"June 21, 1991",1,0,0,1,0,0,0,0,0,0,0,0
84,wingada01,1991,SAS,"June 28, 1991",1,0,0,1,0,0,0,0,0,0,0,0
85,keysra01,1991,CHA,"July 15, 1991",1,0,0,1,0,0,0,0,0,0,0,0


In [24]:
# drop team and date columns from transaction_df

transaction_df = transaction_df.drop(columns=['TEAM', 'DATE'])

In [25]:
# aggregate transactions for player/season pairs

agg_transaction_df = transaction_df.groupby(['PLAYER_ID', 'SEASON_START'], as_index=False).agg('max').reset_index(drop=True)

### Aggregating data for player/season pairs

Now we will aggregate our data for based on PLAYER_ID and SEASON_START. Since we will be interested in using data from a given season to predict whether a player will be moved the following season, we will use the following definitions for some of the features:
- TEAM
    - The team which the player is signed to at the end of the given season.
- Statistical features
    - The total statistics for the entire season.
- SALARY
    - The player's total salary for the given season.
- PCT_TEAM_SALARY
    - The player's total salary for the given season as a percent of TEAM's salary cap for the given season.

In [26]:
agg_stats_df = (
    stats_df
    .groupby(['PLAYER_ID', 'SEASON_START'], as_index=False)[stats_df.columns]
    .apply(lambda group: group[group['TEAM']=='TOT']
           if 'TOT' in group['TEAM'].values
           else group)
    .reset_index(drop=True)
)

In [27]:
agg_stats_df

,PLAYER_ID,SEASON_START,PLAYER_NAME,AGE,TEAM,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,abdelal01,1990,Alaa Abdelnaby,22.0,POR,PF,43.0,0.0,6.7,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2
1,abdelal01,1991,Alaa Abdelnaby,23.0,POR,PF,71.0,1.0,13.2,2.5,5.1,0.493,0.0,0.0,NaN,2.5,5.1,0.493,0.493,1.1,1.4,0.752,1.1,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1,13.5,0.533,0.000,0.280,9.5,20.9,15.2,4.7,1.3,1.1,14.0,20.6,0.6,1.5,2.1,0.110,-2.3,-0.4,-2.6,-0.1
2,abdelal01,1992,Alaa Abdelnaby,24.0,TOT,PF,75.0,52.0,17.5,3.3,6.3,0.518,0.0,0.0,0.00,3.3,6.3,0.519,0.518,1.2,1.5,0.759,1.7,2.8,4.5,0.4,0.3,0.3,1.3,2.5,7.7,13.2,0.551,0.002,0.245,11.0,18.1,14.6,3.0,1.0,1.2,15.6,20.5,0.7,1.3,2.0,0.074,-2.4,-1.5,-3.9,-0.6
3,abdelal01,1993,Alaa Abdelnaby,25.0,BOS,PF,13.0,0.0,12.2,1.8,4.2,0.436,0.0,0.0,NaN,1.8,4.2,0.436,0.436,1.2,1.9,0.640,0.9,2.6,3.5,0.2,0.2,0.2,1.3,1.5,4.9,9.2,0.485,0.000,0.455,8.5,24.2,16.3,2.7,0.6,1.2,20.5,22.6,-0.2,0.1,-0.1,-0.032,-5.3,-2.2,-7.4,-0.2
4,abdelal01,1994,Alaa Abdelnaby,26.0,TOT,PF,54.0,0.0,9.4,2.2,4.3,0.511,0.0,0.0,0.00,2.2,4.2,0.515,0.511,0.4,0.6,0.571,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7,12.6,0.519,0.009,0.152,8.7,17.4,13.1,5.0,1.5,1.8,15.4,25.6,-0.4,0.7,0.3,0.027,-4.4,0.1,-4.3,-0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15827,zubaciv01,2019,Ivica Zubac,22.0,LAC,C,72.0,70.0,18.4,3.3,5.3,0.613,0.0,0.0,0.00,3.3,5.3,0.616,0.613,1.7,2.3,0.747,2.7,4.8,7.5,1.1,0.2,0.9,0.8,2.3,8.3,21.7,0.651,0.005,0.431,15.9,26.4,21.3,9.1,0.6,4.4,11.8,16.4,4.4,2.3,6.6,0.241,1.9,0.9,2.8,1.6
15828,zubaciv01,2020,Ivica Zubac,23.0,LAC,C,72.0,33.0,22.3,3.6,5.5,0.652,0.0,0.1,0.25,3.6,5.4,0.656,0.654,1.9,2.4,0.789,2.6,4.6,7.2,1.3,0.3,0.9,1.1,2.6,9.0,19.1,0.693,0.010,0.434,13.6,22.4,18.1,7.9,0.7,3.4,14.7,15.1,4.8,2.1,6.9,0.206,0.6,0.4,1.0,1.2
15829,zubaciv01,2021,Ivica Zubac,24.0,LAC,C,76.0,76.0,24.4,4.1,6.5,0.626,0.0,0.0,NaN,4.1,6.5,0.626,0.626,2.2,3.0,0.727,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3,19.2,0.660,0.000,0.459,12.8,23.7,18.4,9.8,1.0,3.5,16.1,16.8,4.3,2.9,7.2,0.187,-0.1,0.8,0.7,1.3
15830,zubaciv01,2022,Ivica Zubac,25.0,LAC,C,76.0,76.0,28.6,4.3,6.8,0.634,0.0,0.0,0.00,4.3,6.7,0.637,0.634,2.2,3.1,0.697,3.1,6.8,9.9,1.0,0.4,1.3,1.5,2.9,10.8,16.7,0.661,0.004,0.463,12.3,26.5,19.5,5.1,0.7,4.0,15.9,14.8,3.8,2.9,6.7,0.149,-1.1,0.2,-0.9,0.6


In [28]:
# edit TEAM to be as described above

agg_stats_df['TEAM'] = (
    stats_df
    .groupby(['PLAYER_ID', 'SEASON_START'])['TEAM']
    .agg(list)
    .apply(lambda x: x[-1])
    .reset_index(drop=True)
)

In [29]:
agg_stats_df

,PLAYER_ID,SEASON_START,PLAYER_NAME,AGE,TEAM,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,abdelal01,1990,Alaa Abdelnaby,22.0,POR,PF,43.0,0.0,6.7,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2
1,abdelal01,1991,Alaa Abdelnaby,23.0,POR,PF,71.0,1.0,13.2,2.5,5.1,0.493,0.0,0.0,NaN,2.5,5.1,0.493,0.493,1.1,1.4,0.752,1.1,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1,13.5,0.533,0.000,0.280,9.5,20.9,15.2,4.7,1.3,1.1,14.0,20.6,0.6,1.5,2.1,0.110,-2.3,-0.4,-2.6,-0.1
2,abdelal01,1992,Alaa Abdelnaby,24.0,BOS,PF,75.0,52.0,17.5,3.3,6.3,0.518,0.0,0.0,0.00,3.3,6.3,0.519,0.518,1.2,1.5,0.759,1.7,2.8,4.5,0.4,0.3,0.3,1.3,2.5,7.7,13.2,0.551,0.002,0.245,11.0,18.1,14.6,3.0,1.0,1.2,15.6,20.5,0.7,1.3,2.0,0.074,-2.4,-1.5,-3.9,-0.6
3,abdelal01,1993,Alaa Abdelnaby,25.0,BOS,PF,13.0,0.0,12.2,1.8,4.2,0.436,0.0,0.0,NaN,1.8,4.2,0.436,0.436,1.2,1.9,0.640,0.9,2.6,3.5,0.2,0.2,0.2,1.3,1.5,4.9,9.2,0.485,0.000,0.455,8.5,24.2,16.3,2.7,0.6,1.2,20.5,22.6,-0.2,0.1,-0.1,-0.032,-5.3,-2.2,-7.4,-0.2
4,abdelal01,1994,Alaa Abdelnaby,26.0,PHI,PF,54.0,0.0,9.4,2.2,4.3,0.511,0.0,0.0,0.00,2.2,4.2,0.515,0.511,0.4,0.6,0.571,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7,12.6,0.519,0.009,0.152,8.7,17.4,13.1,5.0,1.5,1.8,15.4,25.6,-0.4,0.7,0.3,0.027,-4.4,0.1,-4.3,-0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15827,zubaciv01,2019,Ivica Zubac,22.0,LAC,C,72.0,70.0,18.4,3.3,5.3,0.613,0.0,0.0,0.00,3.3,5.3,0.616,0.613,1.7,2.3,0.747,2.7,4.8,7.5,1.1,0.2,0.9,0.8,2.3,8.3,21.7,0.651,0.005,0.431,15.9,26.4,21.3,9.1,0.6,4.4,11.8,16.4,4.4,2.3,6.6,0.241,1.9,0.9,2.8,1.6
15828,zubaciv01,2020,Ivica Zubac,23.0,LAC,C,72.0,33.0,22.3,3.6,5.5,0.652,0.0,0.1,0.25,3.6,5.4,0.656,0.654,1.9,2.4,0.789,2.6,4.6,7.2,1.3,0.3,0.9,1.1,2.6,9.0,19.1,0.693,0.010,0.434,13.6,22.4,18.1,7.9,0.7,3.4,14.7,15.1,4.8,2.1,6.9,0.206,0.6,0.4,1.0,1.2
15829,zubaciv01,2021,Ivica Zubac,24.0,LAC,C,76.0,76.0,24.4,4.1,6.5,0.626,0.0,0.0,NaN,4.1,6.5,0.626,0.626,2.2,3.0,0.727,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3,19.2,0.660,0.000,0.459,12.8,23.7,18.4,9.8,1.0,3.5,16.1,16.8,4.3,2.9,7.2,0.187,-0.1,0.8,0.7,1.3
15830,zubaciv01,2022,Ivica Zubac,25.0,LAC,C,76.0,76.0,28.6,4.3,6.8,0.634,0.0,0.0,0.00,4.3,6.7,0.637,0.634,2.2,3.1,0.697,3.1,6.8,9.9,1.0,0.4,1.3,1.5,2.9,10.8,16.7,0.661,0.004,0.463,12.3,26.5,19.5,5.1,0.7,4.0,15.9,14.8,3.8,2.9,6.7,0.149,-1.1,0.2,-0.9,0.6


Now aggregate salary and transaction data, then merge them into `stats_df`.

In [30]:
agg_salary_df = (
    salary_df
    .groupby(['PLAYER_ID', 'SEASON_START'], as_index=False)['SALARY']
    .agg('sum')
    .reset_index(drop=True)
)

agg_transaction_df = (
    transaction_df
    .groupby(['PLAYER_ID', 'SEASON_START'], as_index=False)[transaction_df.columns]
    .agg('max')
    .reset_index(drop=True)
)

In [31]:
agg_nba_df = pd.merge(agg_stats_df, agg_salary_df, how='left', on=['PLAYER_ID', 'SEASON_START'])

agg_nba_df = pd.merge(agg_nba_df, agg_transaction_df, how='left', on=['PLAYER_ID', 'SEASON_START'])

In [32]:
agg_nba_df

,PLAYER_ID,SEASON_START,PLAYER_NAME,AGE,TEAM,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,SALARY,WAIVED_OFF,WAIVED_REG,WAIVED_POST,WAIVED,RELEASED_OFF,RELEASED_REG,RELEASED_POST,RELEASED,TRADED_OFF,TRADED_REG,TRADED_POST,TRADED
0,abdelal01,1990,Alaa Abdelnaby,22.0,POR,PF,43.0,0.0,6.7,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,395000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,abdelal01,1991,Alaa Abdelnaby,23.0,POR,PF,71.0,1.0,13.2,2.5,5.1,0.493,0.0,0.0,NaN,2.5,5.1,0.493,0.493,1.1,1.4,0.752,1.1,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1,13.5,0.533,0.000,0.280,9.5,20.9,15.2,4.7,1.3,1.1,14.0,20.6,0.6,1.5,2.1,0.110,-2.3,-0.4,-2.6,-0.1,494000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abdelal01,1992,Alaa Abdelnaby,24.0,BOS,PF,75.0,52.0,17.5,3.3,6.3,0.518,0.0,0.0,0.00,3.3,6.3,0.519,0.518,1.2,1.5,0.759,1.7,2.8,4.5,0.4,0.3,0.3,1.3,2.5,7.7,13.2,0.551,0.002,0.245,11.0,18.1,14.6,3.0,1.0,1.2,15.6,20.5,0.7,1.3,2.0,0.074,-2.4,-1.5,-3.9,-0.6,500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,abdelal01,1993,Alaa Abdelnaby,25.0,BOS,PF,13.0,0.0,12.2,1.8,4.2,0.436,0.0,0.0,NaN,1.8,4.2,0.436,0.436,1.2,1.9,0.640,0.9,2.6,3.5,0.2,0.2,0.2,1.3,1.5,4.9,9.2,0.485,0.000,0.455,8.5,24.2,16.3,2.7,0.6,1.2,20.5,22.6,-0.2,0.1,-0.1,-0.032,-5.3,-2.2,-7.4,-0.2,805000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abdelal01,1994,Alaa Abdelnaby,26.0,PHI,PF,54.0,0.0,9.4,2.2,4.3,0.511,0.0,0.0,0.00,2.2,4.2,0.515,0.511,0.4,0.6,0.571,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7,12.6,0.519,0.009,0.152,8.7,17.4,13.1,5.0,1.5,1.8,15.4,25.6,-0.4,0.7,0.3,0.027,-4.4,0.1,-4.3,-0.3,650000.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15827,zubaciv01,2019,Ivica Zubac,22.0,LAC,C,72.0,70.0,18.4,3.3,5.3,0.613,0.0,0.0,0.00,3.3,5.3,0.616,0.613,1.7,2.3,0.747,2.7,4.8,7.5,1.1,0.2,0.9,0.8,2.3,8.3,21.7,0.651,0.005,0.431,15.9,26.4,21.3,9.1,0.6,4.4,11.8,16.4,4.4,2.3,6.6,0.241,1.9,0.9,2.8,1.6,6481482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15828,zubaciv01,2020,Ivica Zubac,23.0,LAC,C,72.0,33.0,22.3,3.6,5.5,0.652,0.0,0.1,0.25,3.6,5.4,0.656,0.654,1.9,2.4,0.789,2.6,4.6,7.2,1.3,0.3,0.9,1.1,2.6,9.0,19.1,0.693,0.010,0.434,13.6,22.4,18.1,7.9,0.7,3.4,14.7,15.1,4.8,2.1,6.9,0.206,0.6,0.4,1.0,1.2,7000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15829,zubaciv01,2021,Ivica Zubac,24.0,LAC,C,76.0,76.0,24.4,4.1,6.5,0.626,0.0,0.0,NaN,4.1,6.5,0.626,0.626,2.2,3.0,0.727,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3,19.2,0.660,0.000,0.459,12.8,23.7,18.4,9.8,1.0,3.5,16.1,16.8,4.3,2.9,7.2,0.187,-0.1,0.8,0.7,1.3,7518518.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15830,zubaciv01,2022,Ivica Zubac,25.0,LAC,C,76.0,76.0,28.6,4.3,6.8,0.634,0.0,0.0,0.00,4.3,6.7,0.637,0.634,2.2,3.1,0.697,3.1,6.8,9.9,1.0,0.4,1.3,1.5,2.9,10.8,16.7,0.661,0.004,0.463,12.3,26.5,19.5,5.1,0.7,4.0,15.9,14.8,3.8,2.9,6.7,0.149,-1.1,0.2,-0.9,0.6,10123457.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We still need columns indicating whether a transaction occurs during the next season. We do this by first creating a "next season transaction" DataFrame which is just `transaction_df` with SEASON_START decremented by 1 and the columns renamed, and then merging it into `nba_df` using a left join.

In [33]:
# create DataFrame for "next season transactions"

next_season_agg_transaction_df = agg_transaction_df.rename(columns={'WAIVED_OFF': 'WAIVED_NEXT_OFF',
                                                                    'WAIVED_REG': 'WAIVED_NEXT_REG',
                                                                    'WAIVED_POST': 'WAIVED_NEXT_POST',
                                                                    'WAIVED': 'WAIVED_NEXT',
                                                                    'RELEASED_OFF': 'RELEASED_NEXT_OFF',
                                                                    'RELEASED_REG': 'RELEASED_NEXT_REG',
                                                                    'RELEASED_POST': 'RELEASED_NEXT_POST',
                                                                    'RELEASED': 'RELEASED_NEXT',
                                                                    'TRADED_OFF': 'TRADED_NEXT_OFF',
                                                                    'TRADED_REG': 'TRADED_NEXT_REG',
                                                                    'TRADED_POST': 'TRADED_NEXT_POST',
                                                                    'TRADED': 'TRADED_NEXT'})

next_season_agg_transaction_df['SEASON_START'] -= 1

In [34]:
agg_nba_df = pd.merge(agg_nba_df, next_season_agg_transaction_df, how='left', on=['PLAYER_ID', 'SEASON_START'])

In [35]:
agg_nba_df

,PLAYER_ID,SEASON_START,PLAYER_NAME,AGE,TEAM,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,SALARY,WAIVED_OFF,WAIVED_REG,WAIVED_POST,WAIVED,RELEASED_OFF,RELEASED_REG,RELEASED_POST,RELEASED,TRADED_OFF,TRADED_REG,TRADED_POST,TRADED,WAIVED_NEXT_OFF,WAIVED_NEXT_REG,WAIVED_NEXT_POST,WAIVED_NEXT,RELEASED_NEXT_OFF,RELEASED_NEXT_REG,RELEASED_NEXT_POST,RELEASED_NEXT,TRADED_NEXT_OFF,TRADED_NEXT_REG,TRADED_NEXT_POST,TRADED_NEXT
0,abdelal01,1990,Alaa Abdelnaby,22.0,POR,PF,43.0,0.0,6.7,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,395000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,abdelal01,1991,Alaa Abdelnaby,23.0,POR,PF,71.0,1.0,13.2,2.5,5.1,0.493,0.0,0.0,NaN,2.5,5.1,0.493,0.493,1.1,1.4,0.752,1.1,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1,13.5,0.533,0.000,0.280,9.5,20.9,15.2,4.7,1.3,1.1,14.0,20.6,0.6,1.5,2.1,0.110,-2.3,-0.4,-2.6,-0.1,494000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,abdelal01,1992,Alaa Abdelnaby,24.0,BOS,PF,75.0,52.0,17.5,3.3,6.3,0.518,0.0,0.0,0.00,3.3,6.3,0.519,0.518,1.2,1.5,0.759,1.7,2.8,4.5,0.4,0.3,0.3,1.3,2.5,7.7,13.2,0.551,0.002,0.245,11.0,18.1,14.6,3.0,1.0,1.2,15.6,20.5,0.7,1.3,2.0,0.074,-2.4,-1.5,-3.9,-0.6,500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abdelal01,1993,Alaa Abdelnaby,25.0,BOS,PF,13.0,0.0,12.2,1.8,4.2,0.436,0.0,0.0,NaN,1.8,4.2,0.436,0.436,1.2,1.9,0.640,0.9,2.6,3.5,0.2,0.2,0.2,1.3,1.5,4.9,9.2,0.485,0.000,0.455,8.5,24.2,16.3,2.7,0.6,1.2,20.5,22.6,-0.2,0.1,-0.1,-0.032,-5.3,-2.2,-7.4,-0.2,805000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,abdelal01,1994,Alaa Abdelnaby,26.0,PHI,PF,54.0,0.0,9.4,2.2,4.3,0.511,0.0,0.0,0.00,2.2,4.2,0.515,0.511,0.4,0.6,0.571,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7,12.6,0.519,0.009,0.152,8.7,17.4,13.1,5.0,1.5,1.8,15.4,25.6,-0.4,0.7,0.3,0.027,-4.4,0.1,-4.3,-0.3,650000.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15827,zubaciv01,2019,Ivica Zubac,22.0,LAC,C,72.0,70.0,18.4,3.3,5.3,0.613,0.0,0.0,0.00,3.3,5.3,0.616,0.613,1.7,2.3,0.747,2.7,4.8,7.5,1.1,0.2,0.9,0.8,2.3,8.3,21.7,0.651,0.005,0.431,15.9,26.4,21.3,9.1,0.6,4.4,11.8,16.4,4.4,2.3,6.6,0.241,1.9,0.9,2.8,1.6,6481482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15828,zubaciv01,2020,Ivica Zubac,23.0,LAC,C,72.0,33.0,22.3,3.6,5.5,0.652,0.0,0.1,0.25,3.6,5.4,0.656,0.654,1.9,2.4,0.789,2.6,4.6,7.2,1.3,0.3,0.9,1.1,2.6,9.0,19.1,0.693,0.010,0.434,13.6,22.4,18.1,7.9,0.7,3.4,14.7,15.1,4.8,2.1,6.9,0.206,0.6,0.4,1.0,1.2,7000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15829,zubaciv01,2021,Ivica Zubac,24.0,LAC,C,76.0,76.0,24.4,4.1,6.5,0.626,0.0,0.0,NaN,4.1,6.5,0.626,0.626,2.2,3.0,0.727,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3,19.2,0.660,0.000,0.459,12.8,23.7,18.4,9.8,1.0,3.5,16.1,16.8,4.3,2.9,7.2,0.187,-0.1,0.8,0.7,1.3,7518518.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15830,zubaciv01,2022,Ivica Zubac,25.0,LAC,C,76.0,76.0,28.6,4.3,6.8,0.634,0.0,0.0,0.00,4.3,6.7,0.637,0.634,2.2,3.1,0.697,3.1,6.8,9.9,1.0,0.4,1.3,1.5,2.9,10.8,16.7,0.661,0.004,0.463,12.3,26.5,19.5,5.1,0.7,4.0,15.9,14.8,3.8,2.9,6.7,0.149,-1.1,0.2,-0.9,0.6,10123457.0,NaN,NaN,NaN

Now we fill all of the null values in the various transaction columns with 0.

In [36]:
# define transaction_columns, fill null values in transaction columns with 0

transaction_columns = ['WAIVED_OFF', 'WAIVED_REG', 'WAIVED_POST', 'WAIVED',
                       'RELEASED_OFF', 'RELEASED_REG', 'RELEASED_POST', 'RELEASED',
                       'TRADED_OFF', 'TRADED_REG', 'TRADED_POST', 'TRADED',
                       'WAIVED_NEXT_OFF', 'WAIVED_NEXT_REG', 'WAIVED_NEXT_POST', 'WAIVED_NEXT',
                       'RELEASED_NEXT_OFF', 'RELEASED_NEXT_REG', 'RELEASED_NEXT_POST', 'RELEASED_NEXT',
                       'TRADED_NEXT_OFF', 'TRADED_NEXT_REG', 'TRADED_NEXT_POST', 'TRADED_NEXT']

agg_nba_df[transaction_columns] = agg_nba_df[transaction_columns].fillna(0)

In [37]:
agg_nba_df

,PLAYER_ID,SEASON_START,PLAYER_NAME,AGE,TEAM,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,SALARY,WAIVED_OFF,WAIVED_REG,WAIVED_POST,WAIVED,RELEASED_OFF,RELEASED_REG,RELEASED_POST,RELEASED,TRADED_OFF,TRADED_REG,TRADED_POST,TRADED,WAIVED_NEXT_OFF,WAIVED_NEXT_REG,WAIVED_NEXT_POST,WAIVED_NEXT,RELEASED_NEXT_OFF,RELEASED_NEXT_REG,RELEASED_NEXT_POST,RELEASED_NEXT,TRADED_NEXT_OFF,TRADED_NEXT_REG,TRADED_NEXT_POST,TRADED_NEXT
0,abdelal01,1990,Alaa Abdelnaby,22.0,POR,PF,43.0,0.0,6.7,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,395000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,abdelal01,1991,Alaa Abdelnaby,23.0,POR,PF,71.0,1.0,13.2,2.5,5.1,0.493,0.0,0.0,NaN,2.5,5.1,0.493,0.493,1.1,1.4,0.752,1.1,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1,13.5,0.533,0.000,0.280,9.5,20.9,15.2,4.7,1.3,1.1,14.0,20.6,0.6,1.5,2.1,0.110,-2.3,-0.4,-2.6,-0.1,494000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,abdelal01,1992,Alaa Abdelnaby,24.0,BOS,PF,75.0,52.0,17.5,3.3,6.3,0.518,0.0,0.0,0.00,3.3,6.3,0.519,0.518,1.2,1.5,0.759,1.7,2.8,4.5,0.4,0.3,0.3,1.3,2.5,7.7,13.2,0.551,0.002,0.245,11.0,18.1,14.6,3.0,1.0,1.2,15.6,20.5,0.7,1.3,2.0,0.074,-2.4,-1.5,-3.9,-0.6,500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,abdelal01,1993,Alaa Abdelnaby,25.0,BOS,PF,13.0,0.0,12.2,1.8,4.2,0.436,0.0,0.0,NaN,1.8,4.2,0.436,0.436,1.2,1.9,0.640,0.9,2.6,3.5,0.2,0.2,0.2,1.3,1.5,4.9,9.2,0.485,0.000,0.455,8.5,24.2,16.3,2.7,0.6,1.2,20.5,22.6,-0.2,0.1,-0.1,-0.032,-5.3,-2.2,-7.4,-0.2,805000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,abdelal01,1994,Alaa Abdelnaby,26.0,PHI,PF,54.0,0.0,9.4,2.2,4.3,0.511,0.0,0.0,0.00,2.2,4.2,0.515,0.511,0.4,0.6,0.571,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7,12.6,0.519,0.009,0.152,8.7,17.4,13.1,5.0,1.5,1.8,15.4,25.6,-0.4,0.7,0.3,0.027,-4.4,0.1,-4.3,-0.3,650000.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15827,zubaciv01,2019,Ivica Zubac,22.0,LAC,C,72.0,70.0,18.4,3.3,5.3,0.613,0.0,0.0,0.00,3.3,5.3,0.616,0.613,1.7,2.3,0.747,2.7,4.8,7.5,1.1,0.2,0.9,0.8,2.3,8.3,21.7,0.651,0.005,0.431,15.9,26.4,21.3,9.1,0.6,4.4,11.8,16.4,4.4,2.3,6.6,0.241,1.9,0.9,2.8,1.6,6481482.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15828,zubaciv01,2020,Ivica Zubac,23.0,LAC,C,72.0,33.0,22.3,3.6,5.5,0.652,0.0,0.1,0.25,3.6,5.4,0.656,0.654,1.9,2.4,0.789,2.6,4.6,7.2,1.3,0.3,0.9,1.1,2.6,9.0,19.1,0.693,0.010,0.434,13.6,22.4,18.1,7.9,0.7,3.4,14.7,15.1,4.8,2.1,6.9,0.206,0.6,0.4,1.0,1.2,7000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15829,zubaciv01,2021,Ivica Zubac,24.0,LAC,C,76.0,76.0,24.4,4.1,6.5,0.626,0.0,0.0,NaN,4.1,6.5,0.626,0.626,2.2,3.0,0.727,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3,19.2,0.660,0.000,0.459,12.8,23.7,18.4,9.8,1.0,3.5,16.1,16.8,4.3,2.9,7.2,0.187,-0.1,0.8,0.7,1.3,7518518.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15830,zubaciv01,2022,Ivica Zubac,25.0,LAC,C,76.0,76.0,28.6,4.3,6.8,0.634,0.0,0.0,0.00,4.3,6.7,0.637,0.634,2.2,3.1,0.697,3.1,6.8,9.9,1.0,0.4,1.3,1.5,2.9,10.8,16.7,0.661,0.004,0.463,12.3,26.5,19.5,5.1,0.7,4.0,15.9,14.8,3.8,2.9,6.7,0.149,-1.1,0.2,-0.9,0.6,10123457.0,0.0,0.0,0.0

Let's check how many rows in `agg_nba_df` have missing salary data.

In [38]:
len(agg_nba_df)

15832

In [39]:
agg_nba_df[agg_nba_df['SALARY'].isnull()]

,PLAYER_ID,SEASON_START,PLAYER_NAME,AGE,TEAM,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,SALARY,WAIVED_OFF,WAIVED_REG,WAIVED_POST,WAIVED,RELEASED_OFF,RELEASED_REG,RELEASED_POST,RELEASED,TRADED_OFF,TRADED_REG,TRADED_POST,TRADED,WAIVED_NEXT_OFF,WAIVED_NEXT_REG,WAIVED_NEXT_POST,WAIVED_NEXT,RELEASED_NEXT_OFF,RELEASED_NEXT_REG,RELEASED_NEXT_POST,RELEASED_NEXT,TRADED_NEXT_OFF,TRADED_NEXT_REG,TRADED_NEXT_POST,TRADED_NEXT
54,adamsja01,2020,Jaylen Adams,24.0,MIL,PG,7.0,0.0,2.6,0.1,1.1,0.125,0.0,0.3,0.000,0.1,0.9,0.167,0.125,0.0,0.0,NaN,0.0,0.4,0.4,0.3,0.0,0.0,0.0,0.1,0.3,-6.5,0.125,0.250,0.000,0.0,16.9,8.8,12.7,0.0,0.0,0.0,18.6,-0.1,0.0,-0.1,-0.252,-15.1,-4.6,-19.8,-0.1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,adamsmi01,1995,Michael Adams,33.0,CHA,PG,21.0,3.0,15.7,1.8,4.0,0.446,0.7,2.0,0.341,1.1,2.0,0.548,0.530,1.2,1.7,0.743,0.2,0.8,1.0,3.2,1.0,0.2,1.2,1.2,5.4,16.1,0.579,0.494,0.422,1.8,6.4,4.1,30.4,3.3,0.9,20.3,17.0,0.6,0.2,0.8,0.117,1.6,1.2,2.8,0.4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
112,ahearbl01,2007,Blake Ahearn,23.0,MIA,PG,12.0,0.0,14.8,1.3,4.8,0.263,0.8,2.8,0.294,0.4,1.9,0.217,0.351,2.5,2.6,0.968,0.1,1.5,1.6,1.6,0.5,0.0,1.3,1.7,5.8,9.0,0.495,0.596,0.544,0.7,12.4,6.4,17.2,1.8,0.0,18.5,23.2,-0.1,0.1,0.0,-0.006,-3.5,-1.3,-4.8,-0.1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,ahearbl01,2011,Blake Ahearn,27.0,UTA,PG,4.0,0.0,7.5,1.0,3.5,0.286,0.5,2.3,0.222,0.5,1.3,0.400,0.357,0.0,0.0,NaN,0.0,0.5,0.5,0.3,0.0,0.0,1.3,1.0,2.5,-7.3,0.357,0.643,0.000,0.0,7.7,3.8,5.2,0.0,0.0,26.3,28.4,-0.2,0.0,-0.2,-0.315,-12.7,-6.8,-19.5,-0.1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,ajincal01,2013,Alexis Ajinça,25.0,NOP,C,56.0,30.0,17.0,2.4,4.4,0.546,0.0,0.0,0.000,2.4,4.4,0.548,0.546,1.0,1.2,0.836,1.7,3.3,4.9,0.7,0.4,0.8,1.1,3.3,5.9,14.6,0.589,0.004,0.269,11.3,22.7,16.9,6.6,1.3,4.1,18.4,16.3,1.2,0.9,2.1,0.104,-2.7,0.1,-2.6,-0.1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15675,wrighch02,2012,Chris Wright,23.0,DAL,SG,3.0,0.0,1.3,0.3,0.7,0.500,0.0,0.0,NaN,0.3,0.7,0.500,0.500,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.7,-1.9,0.500,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,33.3,34.2,0.0,0.0,0.0,-0.375,-10.8,-8.5,-19.2,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15696,wrighho02,1990,Howard Wright,23.0,DAL,PF,15.0,0.0,10.9,1.3,3.1,0.404,0.0,0.1,0.000,1.3,3.1,0.413,0.404,1.1,1.6,0.667,0.8,2.2,3.0,0.2,0.3,0.3,0.7,1.9,3.6,8.7,0.469,0.021,0.511,7.8,22.6,15.0,2.5,1.2,1.8,16.0,17.1,-0.1,0.2,0.1,0.023,-4.9,-0.6,-5.5,-0.1,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15735,youngda01,1992,Danny Young,30.0,DET,PG,65.0,2.0,12.9,1.1,2.6,0.413,0.3,1.0,0.324,0.7,1.5,0.475,0.479,0.4,0.5,0.875,0.2,0.5,0.7,1.8,0.5,0.1,0.5,0.6,2.9,10.4,0.519,0.407,0.192,1.6,4.7,3.1,19.2,1.9,0.4,14.2,10.8,0.9,0.5,1.4,0.081,-1.5,1.0,-0.5,0.3,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15809,zhizhwa01,2000,Wang Zhizhi,23.0,DAL,C,5.0,0.0,7.6,1.6,3.8,0.421,0.0,0.4,0.000,1.6,3.4,0.471,0.421,1.6,2.0,0.800,0.2,1.2,1.4,0.0,0.0,0.0,0.2,1.6,4.8,14.9,0.513,0.105,0.526,3.0,17.2,10.3,0.0,0.0,0.0,4.1,29.2,0.1,0.0,0.1,0.122,-4.6,-4.2,-8.8,-0.1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [40]:
print('Percentage of data with missing salary data:', len(agg_nba_df[agg_nba_df['SALARY'].isnull()])/len(agg_nba_df))

Percentage of data with missing salary data: 0.04554067710965134


### Checking whether a player is in the league next season

Lastly, we add the IN_LEAGUE_NEXT column. Note that all players in the most recent season will have 0 in this column.

In [41]:
# create a set consisting of all (PLAYER_ID, SEASON_START) pairs in merged_data_collapsed_teams

player_season_set = set(agg_nba_df[['PLAYER_ID', 'SEASON_START']].itertuples(index=False, name=None))

In [42]:
# define a function which returns whether a given player for a given season shows up in the dataset for the following season

def in_league_next(row):
    return 1.0 * ((row['PLAYER_ID'], row['SEASON_START'] + 1) in player_season_set)

In [43]:
# apply the above function to the rows of merged_data_collapsed_teams to get IN_LEAGUE_NEXT column

agg_nba_df['IN_LEAGUE_NEXT'] = (
    agg_nba_df[['PLAYER_ID', 'SEASON_START']]
    .apply(in_league_next, axis=1)
)

Let's check this on a single player.

In [44]:
agg_nba_df[agg_nba_df['PLAYER_ID']=='scottby01']

,PLAYER_ID,SEASON_START,PLAYER_NAME,AGE,TEAM,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,SALARY,WAIVED_OFF,WAIVED_REG,WAIVED_POST,WAIVED,RELEASED_OFF,RELEASED_REG,RELEASED_POST,RELEASED,TRADED_OFF,TRADED_REG,TRADED_POST,TRADED,WAIVED_NEXT_OFF,WAIVED_NEXT_REG,WAIVED_NEXT_POST,WAIVED_NEXT,RELEASED_NEXT_OFF,RELEASED_NEXT_REG,RELEASED_NEXT_POST,RELEASED_NEXT,TRADED_NEXT_OFF,TRADED_NEXT_REG,TRADED_NEXT_POST,TRADED_NEXT,IN_LEAGUE_NEXT
12799,scottby01,1990,Byron Scott,29.0,LAL,SG,82.0,82.0,32.1,6.1,12.8,0.477,0.9,2.7,0.324,5.2,10.1,0.517,0.510,1.4,1.8,0.797,0.7,2.3,3.0,2.2,1.2,0.3,1.0,1.8,14.5,14.2,0.534,0.208,0.141,2.5,8.1,5.4,10.3,1.8,0.5,7.1,19.9,3.6,2.9,6.6,0.120,0.3,0.6,0.9,2.0,1100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12800,scottby01,1991,Byron Scott,30.0,LAL,SG,82.0,82.0,32.7,5.6,12.3,0.458,0.7,1.9,0.344,5.0,10.3,0.479,0.485,3.0,3.5,0.838,0.9,2.9,3.8,2.8,1.3,0.3,1.5,1.7,14.9,15.5,0.537,0.156,0.290,3.1,10.2,6.6,13.4,2.0,0.6,9.5,20.4,3.9,2.1,6.0,0.107,1.0,0.1,1.0,2.0,1100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12801,scottby01,1992,Byron Scott,31.0,LAL,SG,58.0,53.0,28.9,5.1,11.4,0.449,0.8,2.3,0.326,4.3,9.0,0.481,0.483,2.7,3.2,0.848,0.5,1.8,2.3,2.7,0.9,0.2,1.2,1.7,13.7,14.4,0.535,0.205,0.279,1.8,7.3,4.6,14.2,1.6,0.5,8.6,20.7,2.4,0.9,3.3,0.094,0.3,-1.0,-0.7,0.5,1100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
12802,scottby01,1993,Byron Scott,32.0,IND,SG,67.0,2.0,17.9,3.8,8.2,0.467,0.4,1.1,0.365,3.4,7.1,0.483,0.492,2.3,2.9,0.805,0.3,1.4,1.6,2.0,0.9,0.1,1.5,1.2,10.4,17.6,0.549,0.135,0.356,1.9,8.5,5.3,18.9,2.7,0.5,14.0,27.0,1.7,1.4,3.1,0.123,1.3,0.2,1.5,1.1,650000.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12803,scottby01,1994,Byron Scott,33.0,IND,PG,80.0,1.0,19.1,3.3,7.3,0.455,1.0,2.5,0.389,2.3,4.8,0.489,0.522,2.4,2.8,0.850,0.2,1.7,1.9,1.4,0.8,0.2,1.5,1.5,10.0,15.8,0.587,0.348,0.389,1.5,10.3,6.0,12.2,2.1,0.7,14.8,24.0,2.1,1.9,3.9,0.124,1.1,0.3,1.4,1.3,1550000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12804,scottby01,1995,Byron Scott,34.0,MEM,SG,80.0,0.0,23.7,3.4,8.5,0.401,0.9,2.8,0.335,2.5,5.7,0.433,0.456,2.5,3.0,0.835,0.5,1.9,2.4,1.5,0.8,0.3,1.3,1.6,10.2,13.0,0.523,0.327,0.359,2.4,9.8,5.9,11.7,1.8,0.9,11.3,21.3,1.0,1.3,2.3,0.059,-0.8,-0.6,-1.3,0.3,1650000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12805,scottby01,1996,Byron Scott,35.0,LAL,SG,79.0,8.0,18.2,2.1,4.8,0.430,0.9,2.4,0.388,1.1,2.4,0.471,0.526,1.6,1.9,0.841,0.3,1.2,1.5,1.3,0.6,0.2,0.7,0.9,6.7,13.5,0.590,0.496,0.398,1.7,7.6,4.7,10.7,1.7,0.8,10.6,15.5,2.8,1.3,4.1,0.137,0.6,0.4,1.0,1.1,247500.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finally, we save our DataFrame as a CSV file.

In [45]:
agg_nba_df.to_csv('./bbref_nba_data.csv', index=False)